In [ ]:
try:
    import resnest
except ModuleNotFoundError:
    !pip install -q "../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest"
    
!pip install -q ../input/timm-pytorch-image-models/pytorch-image-models-master/

In [ ]:
import numpy as np
import librosa as lb
import soundfile as sf
import pandas as pd
import cv2
from pathlib import Path
import re

import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

import time
import timm
import cv2
from resnest.torch import resnest50

# Configs

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

# Data

In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec

In [ ]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

In [ ]:
class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
#         images = cv2.resize(images,(128,281))
        
        
        return images
    
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

In [ ]:
values = df_train['primary_label'].value_counts()
birdname_groups = {}
birdid_groups = {}
for k,v in values.items():
    if v <= 32:
        birdname_groups[k] = 0
        birdid_groups[LABEL_IDS[k]] = 0
    elif v<=64:
        birdname_groups[k] = 1
        birdid_groups[LABEL_IDS[k]] = 1
    elif v<=128:
        birdname_groups[k] = 2
        birdid_groups[LABEL_IDS[k]] = 2
    elif v<=256:
        birdname_groups[k] = 3
        birdid_groups[LABEL_IDS[k]] = 3
    elif v<=512:
        birdname_groups[k] = 4
        birdid_groups[LABEL_IDS[k]] = 4

# Inference

In [ ]:
test_data = BirdCLEFDataset(data=data)
len(test_data), test_data[0].shape

In [ ]:
def load_net(checkpoint_path, num_classes=NUM_CLASSES, model='resnest50'):
    if 'resnest50' in checkpoint_path:
        net = resnest50(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    elif "densenet" in checkpoint_path:
        net = timm.create_model("densenet121", pretrained=False,num_classes=num_classes)
    elif "b0" in checkpoint_path:
        net = timm.create_model('tf_efficientnet_b0_ns', pretrained=False, num_classes=num_classes)
    else:
        net = timm.create_model('tf_efficientnet_b3_ns', pretrained=False, num_classes=num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "").replace("encoder.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

In [ ]:
@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

In [ ]:
@torch.no_grad()
def get_group_thresh_preds(out, group_thresh=[0.49,0.49,0.49,0.49]):
    o = (-out).argsort(1)
    out = out.cpu().numpy()
    o = o.cpu().numpy()
    thresh = min(group_thresh)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo,npred,ou in zip(o,npreds,out):
        p = [] 
        for i in range(npred.item()):
            t = oo[i]
            tt = birdid_groups[t]
            if ou[t] > group_thresh[tt]:
                p.append(t)
        preds.append(p)
    return preds

In [ ]:
len(LABEL_IDS)

In [ ]:
sites = ["COR","COL","SNE","SSW"]
# site_group_thresh ={
#     "COR":[0.47,0.47,0.47,0.47,0.47],
#     "COL":[0.47,0.47,0.47,0.47,0.47],
#     "SNE":[0.47,0.47,0.47,0.47,0.47],
#     "SSW":[0.47,0.47,0.47,0.47,0.47],
# }
site_bird_group = {
    "COR":np.zeros(len(LABEL_IDS),dtype = int),
    "COL":np.zeros(len(LABEL_IDS),dtype = int),
    "SNE":np.zeros(len(LABEL_IDS),dtype = int),
    "SSW":np.zeros(len(LABEL_IDS),dtype = int),
}

bird_info = np.load("../input/sites-bird-num/bird_count800.npy")

for k,(site_bird_info) in enumerate(bird_info):
    for i,(bird_kind) in enumerate(site_bird_info):
        if bird_kind <= 0:
            site_bird_group[sites[k]][i] = 0
        elif bird_kind <= 16:
            site_bird_group[sites[k]][i] = 1
        elif bird_kind <= 32:
            site_bird_group[sites[k]][i] = 2
        elif bird_kind <= 64:
            site_bird_group[sites[k]][i] = 3
        else:
            site_bird_group[sites[k]][i] = 4

In [ ]:
bird_info[0]

In [ ]:
@torch.no_grad()
def get_site_group_thresh_preds(out,site_group_thresh,name = 'COR'):
    o = (-out).argsort(1)
    out = out.cpu().numpy()
    o = o.cpu().numpy()
    thresh = 0.3
    npreds = (out > thresh).sum(1)
    preds = []
    for oo,npred,ou in zip(o,npreds,out):
        p = [] 
        for i in range(npred.item()):
            t = oo[i]
            tt = site_bird_group[name][t]
            if ou[t] > site_group_thresh[name][tt]:
                p.append(t)
        preds.append(p)
    return preds

In [ ]:
def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names

In [ ]:
def predict(nets, test_data, names=True, p=1):
    preds = []
        
    with torch.no_grad():

        for idx in tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = []
            for i in range(len(nets)):
                o = nets[i](xb)
                pred.append(torch.sigmoid(o).cpu().numpy()**p)

#                 if names:
#                     pred = get_bird_names(get_thresh_preds(pred))

            preds.append(pred)
    
    return np.array(preds)
    
def weighted_predict(preds, nets, weights=None):
    if weights is None:
        weights = np.ones((len(nets,)))
        weights /= len(nets)
    
    weighted_pred = np.zeros_like(preds[:,0])
    for i in range(len(nets)):
        weighted_pred += preds[:,i]*weights[i]
        
    return weighted_pred

In [ ]:
def gen_file_name(data):
    file_name = []
    for row in data.itertuples(False):
        site_params = row.filename.split('_')
        site_name = site_params[1]
#         for i in range(num):
        file_name.append(site_name)
    return file_name

In [ ]:
def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

In [ ]:
checkpoint_paths = [
    
    Path("../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth"),
    
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold0_epoch_28_f1_val_6962_20210525190516.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold0_epoch_35_f1_val_6890_20210525192227.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold1_epoch_41_f1_val_7025_20210525213011.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold1_epoch_44_f1_val_7002_20210525213737.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold2_epoch_35_f1_val_6997_20210525231559.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold2_epoch_38_f1_val_6964_20210525232335.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold3_epoch_38_f1_val_6732_20210526013148.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold3_epoch_39_f1_val_6721_20210526013422.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold4_epoch_47_f1_val_6992_20210526040142.pth"),
    Path("../input/bird-b0/birdclef_tf_efficientnet_b0_ns_fold4_epoch_49_f1_val_7020_20210526040645.pth"),
    
    
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold0_epoch_34_f1_val_6977_20210526131127.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold0_epoch_43_f1_val_6979_20210526134218.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold1_epoch_29_f1_val_6898_20210526154558.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold1_epoch_34_f1_val_6939_20210526160315.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold2_epoch_34_f1_val_6849_20210526213431.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold2_epoch_39_f1_val_6822_20210526215139.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold3_epoch_41_f1_val_6805_20210527005346.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold3_epoch_42_f1_val_6763_20210527005717.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold4_epoch_23_f1_val_6827_20210527024520.pth'),
    Path('../input/bird-b3/birdclef_tf_efficientnet_b3_ns_fold4_epoch_37_f1_val_6827_20210527033438.pth'),
    
    Path('../input/resnest50-5folds/birdclef_resnest50_fold0_epoch_27_f1_val_0.69676_20210525184602.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold0_epoch_42_f1_val_0.701555_20210525220354.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold1_epoch_40_f1_val_0.70515_20210526082159.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold1_epoch_43_f1_val_0.70533_20210526090108.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold2_epoch_19_f1_val_0.7038_20210526143731.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold2_epoch_34_f1_val_0.69693_20210526175322.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold3_epoch_23_f1_val_0.70365_20210527021504.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold3_epoch_29_f1_val_0.7054_20210527033125.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold4_epoch_34_f1_val_0.7007_20210527152318.pth'),
    Path('../input/resnest50-5folds/birdclef_resnest50_fold4_epoch_43_f1_val_0.69998_20210527180634.pth'),
    
    Path('../input/densent121-2fold/birdclef_densenet121_fold0_epoch_34_f1_val_0.68485_20210523162917.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold0_epoch_44_f1_val_0.68239_20210523183252.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold1_epoch_36_f1_val_0.69548_20210524030526.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold1_epoch_39_f1_val_0.6893_20210524034203.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold2_epoch_34_f1_val_0.7000853174_20210524125933.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold2_epoch_40_f1_val_0.70008_20210524141448.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold3_epoch_48_f1_val_0.68559_20210525020059.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold3_epoch_49_f1_val_0.68634_20210525021302.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold4_epoch_24_f1_val_0.69277_20210525071727.pth'),
    Path('../input/densent121-2fold/birdclef_densenet121_fold4_epoch_31_f1_val_0.68673_20210525084536.pth'),
]

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]

In [ ]:
raw_pred_probas = predict(nets, test_data, names=False, p=0.25)

In [ ]:
raw_pred_probas.shape

In [ ]:
weights = [2]+[1/10]*10+[3/10]*10+[3/10]*10+[1/10]*10
weights /= np.sum(weights)
pred_probas = weighted_predict(raw_pred_probas, nets, weights=weights)

In [ ]:
pred_probas.shape

In [ ]:
pred_probas_new = []

for i in range(len(pred_probas)):
    p = np.zeros_like(pred_probas[i]) #(120, nclass)
    for j in range(len(pred_probas[i])):
        if j == 0:
            p[j] = pred_probas[i][j]*0.6+pred_probas[i][j+1]*0.4
        elif j == len(pred_probas[i])-1:
            p[j] = pred_probas[i][j]*0.6+pred_probas[i][j-1]*0.4
        else:
            p[j] = pred_probas[i][j]*0.6+pred_probas[i][j-1]*0.2+pred_probas[i][j+1]*0.2
        
    pred_probas_new.append(torch.tensor(p))

In [ ]:
site_info = gen_file_name(data)
print(len(site_info))

In [ ]:

# preds = [get_bird_names(get_thresh_preds(pred, thresh=0.49)) for pred in pred_probas_new]
# preds = [get_bird_names(get_group_thresh_preds(pred,[0.47,0.47,0.47,0.47,0.47])) for pred in pred_probas_new]

site_group_thresh ={
    "COR":[0.53,0.55,0.49,0.41,0.45],
    "COL":[0.53,0.50,0.49,0.45,0.46],
    "SNE":[0.53,0.50,0.49,0.45,0.45],
    "SSW":[0.53,0.47,0.49,0.49,0.47],
}
preds = [get_bird_names(get_site_group_thresh_preds(pred,site_group_thresh,site)) for pred,site in zip(pred_probas_new,site_info)]

# preds = [get_bird_names(get_site_group_thresh_preds(pred,site)) for pred,site in zip(pred_probas_new,site_info)]

In [ ]:
len(preds[0])

In [ ]:
sub = preds_as_df(data, preds)

sub

In [ ]:
sub.to_csv("submission.csv", index=False)

# Small validation

In [ ]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}

In [ ]:
# best_f1 = 0 
# best_th = 0

# if TARGET_PATH:
    
#     for th in np.arange(0.35, 0.65, 0.02):
# #         [th,th,th,th,th],
# #         [0.47,0.47,0.47,0.47,0.47]
#         site_group_thresh ={
#             "COR":[0.53,0.55,0.49,0.41,0.45],
#             "COL":[0.53,0.50,0.49,0.45,0.46],
#             "SNE":[0.53,0.50,0.49,0.45,0.45],
#             "SSW":[0.53,0.47,0.49,0.49,0.47],
#         }
        
# #         preds = [get_bird_names(get_thresh_preds(pred, thresh=th)) for pred in pred_probas_new]
#         preds = [get_bird_names(get_site_group_thresh_preds(pred,site_group_thresh,site)) for pred,site in zip(pred_probas_new,site_info)]

#         sub = preds_as_df(data, preds)

#         sub_target = pd.read_csv(TARGET_PATH)
#         sub_target = sub_target.merge(sub, how="left", on="row_id")

#         df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])

#         f1 = df_metrics.mean().values[0]
        
#         if f1 > best_f1:
#             best_f1 = f1
#             best_th = th
        
#         print(round(th,3), round(f1,7))

        
#     print("best_f1: {},best_th: {}".format(best_f1,best_th))